In [1]:
from data.data import COVID19India

In [2]:
cin = COVID19India()

In [3]:
df = cin.moh_data(save=False)

In [4]:
df.head()

,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
0,Andhra Pradesh,3,0,0,0
1,Chhattisgarh,1,0,0,0
2,Delhi,26,1,5,1
3,Gujarat,14,0,0,0
4,Haryana,3,14,0,0


df

In [5]:
df.head()

,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
0,Andhra Pradesh,3,0,0,0
1,Chhattisgarh,1,0,0,0
2,Delhi,26,1,5,1
3,Gujarat,14,0,0,0
4,Haryana,3,14,0,0


In [18]:
from pathlib import Path

In [21]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


class InshortsNews():
    def __init__(self, category='national'):
        self.category = category
        self.url = 'https://www.inshorts.com/en/read/' + str(self.category)
        self.request_timeout = 120
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

        self.session = requests.Session()
        retries = Retry(total=5, backoff_factor=0.5, status_forcelist=[502, 503, 504])
        self.session.mount('http://', HTTPAdapter(max_retries=retries))

    def __request(self):
        try:
            response = self.session.get(self.url, timeout=self.request_timeout, headers=self.headers)
            response.raise_for_status()
            content = response.content.decode('utf-8')
            return content
        except Exception as e:
            raise

    def get_news(self):
        content = self.__request()
        soup = BeautifulSoup(content, 'html.parser')
        headings = [text.span.text for text in soup.find_all('div', attrs={'class': 'news-card-title'})]

        newss = [text.text.strip().split('\n\n')[0] for text in
                 soup.find_all('div', attrs={'class': 'news-card-content'})]

        short_by = [' '.join(text.div.text.strip().split('/ \n      ')[0].split()[2:])
                    for text in soup.find_all('div', attrs={'class': 'news-card-title'})]

        times = [text.text.strip().split('\n\n')[1].split('/ \n      ')[1].split(' ')[0:]
                 for text in soup.find_all('div', attrs={'class': 'news-card-title'})]
        times = pd.to_datetime([' '.join(text[3:6] + text[:2]) for text in times])

        data = pd.DataFrame(data=[headings, newss, short_by, times]).T
        data.columns = ['headings', 'news', 'short_by', 'time']
        data.sort_values('time', inplace=True, ascending=False)
        data.reset_index(drop=True, inplace=True)
        data['category'] = self.category
        return data

    def get_all_news(self):
        categories = ['national', 'business', 'sports', 'world', 'politics', 'technology',
                      'startup', 'entertainment', 'miscellaneous', 'hatke', 'science', 'automobile']
        data = pd.DataFrame(columns=['headings', 'news', 'short_by', 'time', 'category'])
        for category in categories:
            data1 = InshortsNews(category).get_news()
            data = pd.concat([data, data1], axis=0)
        data.reset_index(drop=True, inplace=True)
        return data

In [28]:
df11 = InshortsNews().get_news()

In [39]:
news = """
"""

for v in df11.values[:10]:
    news += f"# {v[0]}\n```{v[1]}```\n"

In [40]:
print(news)


# Punjab Police dances to 'Baari Barsi' to spread awareness about COVID-19
```Punjab Police DGP Dinkar Gupta on Saturday shared a video of police officials dancing to a self-composed version of the song 'Baari Barsi' for spreading awareness about COVID-19. In the video, the police personnel advised public to cover their mouth with elbow while coughing and sneezing. The officials also asked people to say 'Sat Sri Akal' instead of shaking hands.```
# Flight with 263 students from coronavirus-hit Rome lands in Delhi
```The special Air India flight carrying 263 Indian students that took off from coronavirus-hit Italy's Rome landed at Delhi airport on Sunday. As per reports, all precautions are being exercised and standard operating procedures (SOPs) for handling passengers are being followed. The coronavirus death toll in Italy has reached 4,825 with total cases rising to 53,578.```
# Number of coronavirus cases in India rises to 324
```The total number of coronavirus cases in India has r

In [24]:
df11

,headings,news,short_by,time,category
0,Punjab Police dances to 'Baari Barsi' to sprea...,Punjab Police DGP Dinkar Gupta on Saturday sha...,Jigyasu,2020-03-22 10:51:00,national
1,Flight with 263 students from coronavirus-hit ...,The special Air India flight carrying 263 Indi...,Jigyasu,2020-03-22 10:43:00,national
2,Number of coronavirus cases in India rises to 324,The total number of coronavirus cases in India...,Daisy Mowke,2020-03-22 10:36:00,national
3,4-year-old girl found negative after coronavir...,Assam Health Minister Himanta Biswa Sarma on S...,Daisy Mowke,2020-03-22 09:45:00,national
4,"Indian Railways cancels over 3,700 trains on S...",The Indian Railways on Friday announced the ca...,Jigyasu,2020-03-22 09:34:00,national
...,...,...,...,...,...
294,Researchers trick Tesla cars into speeding wit...,Researchers at McAfee successfully tricked two...,Pragya Swastik,2020-02-19 18:52:00,automobile
295,Porsche Taycan catches fire and burns inside b...,Porsche has confirmed that one of its new Tayc...,Pragya Swastik,2020-02-19 15:52:00,automobile
296,We brought Chinese parts to UK in suitcases: T...,Tata Motors' British luxury unit Jaguar Land R...,Krishna Veera Vanamali,2020-02-19 15:08:00,automobile
297,Fire me immediately if I fail to turn around N...,Nissan's new CEO Makoto Uchida said he would w...,Anushka Dixit,2020-02-18 18:01:00,automobile


In [27]:
df11[39:59]

,headings,news,short_by,time,category
39,IndiGo grounds 30 planes as govt bars int'l fl...,IndiGo has been forced to ground 30 planes aft...,Kiran Khatri,2020-03-20 20:49:00,business
40,India gets $1-mn materials requests from SAARC...,The Ministry of External Affairs (MEA) said In...,Kiran Khatri,2020-03-20 19:19:00,business
41,Uber's former exec admits stealing self-drivin...,"Anthony Levandowski, Uber's former self-drivin...",Aishwarya,2020-03-20 18:50:00,business
42,Uber's rides down by 70% in coronvirus-hit cit...,Uber CEO Dara Khosrowshahi has revealed that c...,Aishwarya,2020-03-20 18:37:00,business
43,"Investors gain ₹7 lakh cr as Sensex, Nifty tra...",Investors gained ₹7 lakh crore on Friday as Se...,Kiran Khatri,2020-03-20 18:34:00,business
44,"Yes Bank to raise ₹20,000 cr through certifica...","Yes Bank is planning to raise ₹20,000 crore th...",Kiran Khatri,2020-03-20 16:54:00,business
45,We can stay open during lockdowns as we are 'e...,Gaming retail platform GameStop told its emplo...,Aishwarya,2020-03-20 16:45:00,business
46,Mukesh Ambani & family raise stake in RIL; all...,"Mukesh Ambani, his wife Nita Ambani and three ...",Dharna,2020-03-20 16:27:00,business
47,Apollo Tyres' top management to take 15-25% pa...,Apollo Tyres has said its top management will ...,Kiran Khatri,2020-03-20 15:40:00,business
48,Coronavirus pandemic to leave behind a global ...,Billionaire Anand Mahindra has said the corona...,Dharna,2020-03-20 13:08:00,business
